In [1]:
include("setup.jl")

┌ Info: Installing packages. This may take up to a few minutes.
└ @ Main /home/alexis/Bureau/depots_git/tutorial_jump/setup.jl:1


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Info: Loading packages
└ @ Main /home/alexis/Bureau/depots_git/tutorial_jump/setup.jl:7
┌ Info: All packages successfully imported.
└ @ Main /home/alexis/Bureau/depots_git/tutorial_jump/setup.jl:23


# Asymmetric Travelling Salesman Problem

https://www.movable-type.co.uk/scripts/latlong.html
### Problem description

$$
    \begin{array}{cl}
        \min_{x} \ \ \ &
            \sum_{i, j} d_{i, j} x_{i, j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in \Omega\\
        &    \sum_{i} x_{i, j} = 1, \ \ \ \forall j \in \Omega\\
        &    x_{k,k} = 0 \ \ \ \forall k \in \Omega\\
        &   \sum_{i} \sum_{j \ne i} x_{i,j} \le |S| - 1, \forall S \subset \Omega, 2 \le |S| \le N-1\\
        &   x_{i, j} \in \{0, 1\}, \ \ \ \forall (i,j) \in \Omega^2 , i \ne j\\
        & \Omega = \{1, \dots, N\}
    \end{array}
$$

In [2]:
include("tsp/DataReader.jl")

In [3]:
d = load(string(workspace,"/tsp/distances_quebec.jld"),"distances")

9×9 Array{Int64,2}:
   0  256  137  152   193  461   542   627   889
 256    0  125  209   422  206   313   848   719
 137  125    0  143   297  326   424   723   829
 152  209  143    0   337  414   484   776   737
 193  422  297  337     0  623   720   517  1075
 461  206  326  414   623    0   249   840   700
 542  313  424  484   720  249     0  1089   511
 627  848  723  776   517  840  1089     0  1552
 889  719  829  737  1075  700   511  1552     0

In [4]:
c = load(string(workspace,"/tsp/cities_quebec.jld"),"cities")

9×1 Array{SubString{String},2}:
 "Montréal"      
 "Québec"        
 "Trois-Rivières"
 "Sherbrooke"    
 "Hull"          
 "Chicoutimi"    
 "Rimouski"      
 "Rouyn"         
 "Moncton"       

In [5]:
function subtours(x::Vector{T}) where T
    res = Vector{T}[[]]
    for elem in x, j in eachindex(res)
        push!(res, [res[j] ; elem])
    end
    return res
end

subtours (generic function with 1 method)

In [6]:
subtours([1:5;])

32-element Array{Array{Int64,1},1}:
 []             
 [1]            
 [2]            
 [1, 2]         
 [3]            
 [1, 3]         
 [2, 3]         
 [1, 2, 3]      
 [4]            
 [1, 4]         
 [2, 4]         
 [1, 2, 4]      
 [3, 4]         
 ⋮              
 [3, 5]         
 [1, 3, 5]      
 [2, 3, 5]      
 [1, 2, 3, 5]   
 [4, 5]         
 [1, 4, 5]      
 [2, 4, 5]      
 [1, 2, 4, 5]   
 [3, 4, 5]      
 [1, 3, 4, 5]   
 [2, 3, 4, 5]   
 [1, 2, 3, 4, 5]

In [7]:
function solve_tsp(D, optimizer)
    # number of cities
    N = size(D, 1)
    
    # instantiate model
    mip = Model(with_optimizer(optimizer))

    # Basic formulation
    @variable(mip, X[1:N, 1:N], Bin)

    # Objective
    @objective(mip, Min, sum(X.*D)) # sum(D[i,j] * X[i,j] for i=1:N, j=1:N)

    for k in 1:N
        @constraint(mip, X[k,k] == 0.0)  # city `k` cannot follow itself in the tour
    end

    # Each city has one predecessor and one successor
    for i in 1:N
        @constraint(mip, sum(X[i, j] for j in 1:N) == 1.0)
        @constraint(mip, sum(X[j, i] for j in 1:N) == 1.0)
    end
    
    # We only want a cycle of length N.
    tours = subtours([1:N;])
    for st in tours
        T = length(st)
        if 2 ≤ T ≤ N-1
            @constraint(mip, sum(X[st[k],st[k+1]] for k=1:T-1) + X[st[T],st[1]] ≤ T-1)
            @constraint(mip, sum(X[st[k+1],st[k]] for k=1:T-1) + X[st[1],st[T]] ≤ T-1)
        end
    end

    # solve relaxed model
    optimize!(mip)

    # get solution
    return value.(X)
end

solve_tsp (generic function with 1 method)

In [8]:
solve_tsp(d, Cbc.Optimizer)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3274 - 0.00 seconds
Cgl0002I 9 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 930 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 762 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 568 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 278 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 51 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 0 substitutions
Cgl0004I processed model has 984 rows, 72 columns (72 integer (72 of which binary)) and 7243 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 21 integers unsatisfied sum - 7
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 4232 iterations 33
Cbc0038I Solution found of 4232
Cbc0038I Before min

9×9 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

In [9]:
sol = solve_tsp(d, GLPK.Optimizer)

9×9 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

In [10]:
include("tsp/RoadTrip.jl")

RoadTrip (generic function with 1 method)

In [11]:
RoadTrip(9, c, sol)

Montréal
Hull
Rouyn
Trois-Rivières
Québec
Chicoutimi
Rimouski
Moncton
Sherbrooke


In [12]:
d2 = load(string(workspace,"/tsp/distances_usa.jld"),"distances2")

50×50 Array{Float64,2}:
    0.0     337.637   490.244   955.884  …  1076.38    934.692   760.766
  337.637     0.0     294.552   815.26       823.803   756.247   549.565
  490.244   294.552     0.0     521.244     1029.22   1015.7     803.354
  955.884   815.26    521.244     0.0       1450.03   1494.81   1284.23 
 1046.83    768.442   563.855   473.668     1123.35   1243.23   1053.15 
 1194.35    974.112   709.884   338.261  …  1429.45   1535.48   1337.85 
 1214.04    993.371   729.554   353.464     1443.14   1551.39   1354.39 
 1237.17   1025.66    756.36    348.934     1489.57   1595.09   1396.95 
 1335.87   1132.76    859.582   417.102     1594.26   1703.82   1506.36 
 1379.23   1179.47    904.898   451.736     1640.1    1751.21   1554.02 
 1284.09   1101.31    818.981   342.695  …  1611.54   1705.86   1503.94 
 1492.76   1301.22   1023.56    550.322     1760.58   1875.08   1678.44 
 1650.46   1464.77   1185.45    700.409     1914.96   2035.56   1840.33 
    ⋮                      

In [13]:
c2 = load(string(workspace,"/tsp/cities_usa.jld"),"cities2")

50×1 Array{SubString{String},2}:
 "Cape Canaveral, FL"                
 "Okefenokee Swamp, GA"              
 "Fort Sumter, SC"                   
 "Wright Brothers Visitor Center, NC"
 "Lost World Caverns, WV"            
 "Mount Vernon, VA"                  
 "White House, DC"                   
 "Maryland State House, MD"          
 "New Castle Historic District, DE"  
 "Liberty Bell, PA"                  
 "Congress Hall, Cape May, NJ"       
 "Statue of Liberty, NY"             
 "Mark Twain House, Hartford, CT"    
 ⋮                                   
 "Bryce Canyon, UT"                  
 "Craters of the Moon, ID"           
 "Yellowstone, WY"                   
 "Pikes Peak, CO"                    
 "Carlsbad Caverns, NM"              
 "The Alamo, TX"                     
 "Chickasaw, OK"                     
 "Toltec Mounds, AR"                 
 "Graceland, TN"                     
 "Vicksburg, MS"                     
 "French Quarter, New Orleans, LA"   
 "USS Alabama, AL

In [14]:
# Compute sub-tours from tentative tour `X`.
function find_subtours(n, X)
    (n, n) == size(X) || throw(DimensionMismatch(
        "n=$n but X has size $(size(X))."
    ))

    # List of all sub-tours in `X`
    # Empty if `X` is a tour
    sub_tours = Vector{Vector{Int}}()

    # `f[i]` is the city that follows `i` in the tour
    f = zeros(Int, n)
    @inbounds for j in 1:n
        for i in 1:n
            X[i, j] == 1.0 && (f[i] = j; break;)
        end
    end
    
    flags = falses(n)
    num_seen = 0  
    while num_seen < n
        first = argmin(flags)  # first city in current tour
        flags[first] && break  # all cities have been visited
        subtour = [first]
        flags[first] = true
        num_seen += 1
        next = f[first]
        next == first && error("X[$first, $first] = $(X[first, first]).")
        while next != first
            push!(subtour, next)
            flags[next] = 1
            num_seen += 1
            next = f[next]
        end

        length(subtour) == n && return sub_tours  # X is a tour
        # @info "Found sub-tour: $subtour"
        push!(sub_tours, subtour)
    end
    
    @info "Found $(length(sub_tours)) sub-tours."
    return sub_tours

end

find_subtours (generic function with 1 method)

In [15]:
function add_subtour_elimination!(mip, X, s)

    S = length(s)
    @constraint(mip, sum(X[s[k], s[(k+1)]] for k in 1:S-1) + X[s[end], s[1]] <= S-1)
    @constraint(mip, sum(X[s[k+1], s[k]] for k in 1:S-1) + X[s[1], s[end]] <= S-1)

    return nothing
end

add_subtour_elimination! (generic function with 1 method)

In [16]:
function solve_tsp2(n, D, optimizer; itermax = 100)

    # instantiate model
    mip = Model(with_optimizer(optimizer))

    # Basic formulation
    @variable(mip, X[1:n, 1:n], Bin)

    # Objective
    @objective(mip, Min, sum(X.*D))

    for i in 1:n
        @constraint(mip, X[i, i] == 0.0)  # city `i` cannot follow itself in the tour
    end

    # Each city has one predecessor and one successor
    for i in 1:n
        @constraint(mip, sum(X[i, j] for j in 1:n) == 1.0)
        @constraint(mip, sum(X[j, i] for j in 1:n) == 1.0)
    end

    t_start = time()
    num_iter = 0
    while (time() - t_start < 300.0) && (num_iter < itermax)
        num_iter += 1
        # solve relaxed model
        optimize!(mip)

        # get solution
        X_ = value.(X)

        # find sub-tours
        sub_tours = find_subtours(n, X_)

        if length(sub_tours) == 0
            @info "Optimal solution found"
            return X_
        else
            for subtour in sub_tours
                add_subtour_elimination!(mip, X, subtour)
            end
        end
    end
    
    @info "Time limit reached."
    return value.(X)
end

solve_tsp2 (generic function with 1 method)

In [17]:
sol2 = solve_tsp2(50, d2, GLPK.Optimizer)

┌ Info: Found 21 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 17 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 3 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 4 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 2 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 4 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 3 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 2 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 2 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 2 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 2 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 3 sub-tours.
└ @ Main In[14]:41
┌ Info: Found 3 sub-tours.
└ @ Main In[14]:41
┌ Info: Optimal solution found
└ @ Main In[16]:36


50×50 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [18]:
RoadTrip(50, c2, sol2)

Cape Canaveral, FL
Fort Sumter, SC
Wright Brothers Visitor Center, NC
Lost World Caverns, WV
Mount Vernon, VA
White House, DC
Maryland State House, MD
Congress Hall, Cape May, NJ
New Castle Historic District, DE
Liberty Bell, PA
Statue of Liberty, NY
Mark Twain House, Hartford, CT
The Breakers, Newport, RI
USS Constitution, Boston, MA
Acadia National Park, ME
Mount Washington, Bretton Woods, NH
Shelburne Farms, VT
Olympia Entertainment, Detroit, MI
Spring Grove Cemetery, Cincinnati, OH
Mammoth Cave National Park, KY
West Baden Springs Hotel, IN
Gateway Arch, St. Louis, MO
Lincoln Visitor Center, IL
Taliesin, WI
Fort Snelling, MN
Terrace Hill, IA
C. W. Parker Carousel Museum, KS
Ashfall Fossil Bed, NE
Mount Rushmore, SD
Fort Union Trading Post, ND
Yellowstone, WY
Craters of the Moon, ID
Glacier National Park, MT
Hanford Site, WA
Columbia River Gorge, OR
Cable Car Museum, San Francisco, CA
San Andreas Fault, CA
Hoover Dam, NV
Grand Canyon, AZ
Bryce Canyon, UT
Pikes Peak, CO
Carlsbad Cave

![](tsp/solution_usa.png)